# Parte 1: Estudio comparativo de PE usando características locales

## Introdución
En este notebook, se muestran las métricas obtenidas para los diferentes modelos propuestos. Mediante el dataset Household Power Consumption del portal UCI (https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption), estudiaremos la utilidad de cada uno de los mecanismos, y estableceremos cuál es el mecanismo a usar con otros datasets del estado del arte. El objetivo es estudiar el impacto de diferentes estrategias de encoding para ver cuál nos permite obtener mejores resultados en secuencias de larga extensión.

Examinaremos las siguientes alternativas:

| Codificación             | Descripción                                                       |
|-------------------|------------------------------------------------------------------|
| `no_pe`           | Sin codificación posicional, se usan solo los datos de entrada.      |
| `informer`        | Codificación temporal original de Informer.|
| `stats`           | Codificación basada en estadísticas por ventana temporal deslizante, que calcula media, std y valores extremos.        |
| `stats_lags`      | Igual que `stats`, pero incluye lags como contexto local.    |
| `all_pe_weighted` | Combinación de lo anterior, junto a PE fijos y PE aprendibles (LPE), ponderados con pesos normalizados mediante Softmax.   |
| `tpe`             | Codificación temporal haciendo uso de temporal PE, t-PE, para aportar mayor información local. Contiene la información de lags, ventana y otros PE fijos, haciendo uso de pesos aprendidos y normalizados mediante Softmax. |
| `spe`|Alteración del mecanismo de atención para incorporar información estadística sin necesidad de incrustarla en el embedding.|

Tras dichas pruebas, examinaremos la mejor alternativa en base a las métricas y el costo computacional, y probaremos a experimentar si realmente la información añadida tiene impacto en la calidad de los resultados realizando Shuffle del contexto de entrada del encoder en test.

## Positional encodings empleados

Para comprender de manera fundamentada las alternativas propuestas anteriormente, podemos analizar nuestro punto de partida, Informer, y dos de las propuestas tomadas del paper de Habib Irani y Vangelis Metsis (https://arxiv.org/pdf/2502.12370): TPE y SPE.

### Informer: modelo de partida

Es el PE propuesto originalmente por Vaswani et al. (2017). Utiliza funciones sinusoidales para representar la posición:

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d}}}\right)
$$

$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d}}}\right)
$$

Donde cada variable indica:

- **pos**: índice temporal (posición).
- **i**: dimensión del componente del embedding.
- **d**: dimensión total del embedding.

Basándonos simplemente en su formulación, podemos distinguir ya sus ventajas y desventajas. Como ventaja, podemos ver que se trata de una codificación no entrenable, por lo que para unas mismas dimensiones de entrada de datos, el resultado será siempre el mismo, y sus valores no se deben entrenar época a época de manera independiente. Esto lo hace más sencillo de usar, y más eficiente al no requerir parámetros adicionales. Sin embargo, como principal desventaja, estimamos que no permite modelar con suficiente adaptabilidad los períodos no estacionales de datos o con mayor número de irregularidades y valores anómalos.


### Codificación Posicional Aprendible: Learnable PE (LPE)

En este otro enfoque, en lugar de fijar de antemano la codificación, como en la versión tradicional, establecemos que los parámetros son aprendibles por el modelo, y por tanto, requerirán ser ajustados durante las diferentes épocas, aumentando así la complejidad del modelo, pero permitiendo una mayor adaptabilidad a la forma de los datos de entrada.

$$
PE_{\text{learnable}}(pos) = W_{pos}, \quad W_{pos} \in \mathbb{R}^d
$$

Se incorpora al embedding original de entrada de manera aditiva.

$$
X' = X + PE_{\text{learnable}}[pos]
$$

Por tanto, las principales ventajas son la adaptabilidad al conjunto de datos de entrada, la posibilidad de que el propio modelo ajuste los pesos sin necesidad de realizarlo de manera fija e inflexible. Pero, como es notable en su definición, estamos aumentando el número de parámetros necesarios del modelo, por lo que supondrá un coste más elevado de entrenamiento, y además, a diferencia de los senos/cosenos, será necesario reentrenar la estructura de encoding si los datos de entrada tienen las mismas dimensiones pero distinto contenido, ya que la codificación ha de adaptarse a la forma de estos para minimizar el error.


### Codificación Posicional Temporal (T-PE)

Esta codificación está basada en el tradicional seno-coseno, que agrega la ya mencionada información global y geométrica a los datos, pero además, para mejorar la adaptabilidad a los datos, incorpora información semántica de estos. Para ello, utiliza una función de suavizado gaussiano para tratar de encontrar correspondecia entre cada par de posiciones i-j, de manera que se añada información de semejanza entre las posiciones.


$$
PE_{i, 2k} = \sin\left(\frac{i}{10000^{\frac{2k}{d}}}\right), \quad
PE_{i, 2k+1} = \cos\left(\frac{i}{10000^{\frac{2k}{d}}}\right)
$$

Y para la información local:

$$S(i, j) = \exp\left( -\frac{\|x_i - x_j\|^2}{2\sigma^2} \right)$$

Añadiéndose al encoding final de manera aditiva:

$$T\text{-}PE(i) = PE(i) + S(i, j)$$

En resumen, conserva la sencillez de un encoding tradicional, pero con la determinación de similaridad entre posiciones.


SPE (Stochastic Positional Encoding) introduce aleatoriedad controlada para mejorar la generalización y la captación de dependencias relativas.

### Generación Estocástica Posicional 

SPE es un PE que, a diferencia de los anteriores, que bastaba con sumarlos con la información base del embedding, requieren modificar el mecanismo de atención para su correcto aprendizaje. Comienza con una matriz aleatoria con entradas gaussianas estándar i.i.d.:

$$
Z_d \in \mathbb{R}^{M \times R}
$$

Luego, aplica convoluciones para generar las codificaciones:

$$
Q_d = Z_d \otimes \Phi_Q, \quad K_d = Z_d \otimes \Phi_K
$$

donde:
- **$\otimes$** representa la operación de convolución.
- **( $\Phi_Q, \Phi_K$)** son filtros convolucionales aprendibles.

La atención se calcula como:

$$
P_d \approx \frac{\bar{Q}_d \bar{K}_d^\top}{R}
$$

Este producto aproximado retiene relaciones posicionales mediante convoluciones, y añade complejidad lineal que depende de la longitud de la secuencia.

Sobre el papel, presenta varias ventajas:
- Captura relaciones relativas sin requerir atención cuadrática.
- Permite una mayor escalabilidad gracias a su menor costo computacional
- Es más robusta ante sobreajuste por su componente estocástico, y podría ser eficaz en clasificación de series temporales largas y multivariadas.

Sin embargo, aunque teóricamente pueda parecer más eficiente, modificar el mecanismo de atención puede ser crítico en función del conjunto de datos, ya que al no depender de estos, la codificación podría ser demasiado agnóstica del contexto, y ofrecer peores resultados.

Una vez explicadas los modelos, pasaremos a probarlo sobre el conjunto de datos HPC, tomando como baseline el modelo Informer sin ningún tipo de encoding, es decir, empleando únicamente para la construcción del embedding la información proporcionada directamente por los datos.

## Evaluación de los 7 modelos de PE mediante HPC

Comenzaremos experimentando primero cuál de las 7 alternativas propuestas permite obtener un mejor resultados. En muchas de ellas, como ya se comentaba, se utilizan diversas estrategias de encoding ponderadas, ya que así el modelo puede decidir cuál de ellas es la que mejor se adapta al problema. Esto se debe a que, en ocasiones, la coexistencia de varios métodos no tiene por qué empeorar el resultado, ya que cada problema requiere unas necesidades específicas, en función de la correlación de los datos, y no existe una codificación de PE absoluta mejor en todos los casos.

A continuación, procederemos a la lectura de los ficheros csv que contienen los resultados, y compararemos tanto en tabla como gráficamente sus resultados. Nos fijaremos especialmente en MAE Y MSE, recurriendo al resto de métricas cuando no exista una diferencia clara.

Para tratar de no favorecer algoritmos concretos, todos serán evaluados siguiendo los parámetros aquí reflejados:

- Ventana: 60
- Longitud de secuencia: 180
- Longitud de contexto: 60
- Longitud de predicción: 60 

### Comparativa


In [ ]:
import pandas as pd

# Ficheros con sus nombres de modelo
model_files = {
    "WinStatFlex": "Experimentos/HPC_ALL_PE_v2.csv",
    "No PE": "Experimentos/HPC_No_PE.csv",
    "PE (sin/cos)": "Experimentos/metricas_InformerVanilla_hpc_pe_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebfixed_dtTrue_mxTrue_test_hpc_pe.csv",
    #"ALL_PE_Weighted (No Norm)": "Experimentos/HPC_ALL_PE_v2_NoNorm.csv",
    "Informer": "Experimentos/metricas_InformerVanilla_hpc_informer_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_hpc_informer.csv",
    "Informer SPE": "Experimentos/HPC_SPE.csv",
    "WinStatSPE": "Experimentos/HPC_SPE_ALL.csv",
    "WinStat": "Experimentos/metricas_InformerVanilla_stats_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_stats.csv",
    "WinStatTPE": "Experimentos/metricas_InformerVanilla_tpe_weighted_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tpe_weighted.csv",
    "WinStatLag": "Experimentos/HPC_Ventana_Lags.csv"
}

dfs = []

for model_name, file_path in model_files.items():
    df = pd.read_csv(file_path)
    # print(file_path)
    df['Modelo'] = model_name  # Añadir columna del modelo

    # Pivotar valores de Mean y STD
    df_mean = df.pivot(index='Modelo', columns='Metric', values='Mean')
    df_std = df.pivot(index='Modelo', columns='Metric', values='STD')

    # Renombrar columnas para distinguir Mean y STD
    df_mean.columns = [f"{col}_Mean" for col in df_mean.columns]
    df_std.columns = [f"{col}_STD" for col in df_std.columns]

    # Combinar ambos
    df_combined = pd.concat([df_mean, df_std], axis=1)

    dfs.append(df_combined)

# Unir todos los modelos
final_df = pd.concat(dfs)

# Resetear índice y ordenar por MSE_Mean
final_df.reset_index(inplace=True)
final_df = final_df.sort_values(by="MSE_Mean")

# Mostrar
display(final_df)

Podemos apreciar que el modelo con mejor rendimiento es el basado en varios encodings posicionales los cuales han sido normalizados y ponderados con pesos entre 0 y 1 mediante SoftMax. Son los que mejores resultados ofrecen en MSE, pero también está dentro del top cuando observamos MAE o RMSE, pero ahí si existen leves mejoras en otros modelos. Sin embargo, dado a la cercanía de estas otras alternativas, y a su mayor coste computacional, podríamos considerar que el modelo propuesto en la primera fila es aquel que debemos comparar y verificar con respecto al modelo original.

Para ver los resultados de forma gráfica, podemos representar los valores de MSE y MAE en un gráfico de barras.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_metricas_con_error(df,
                            modelos_col='Modelo',
                            metricas=['MAE_Mean', 'MSE_Mean'],
                            stds=['MAE_STD', 'MSE_STD'],
                            titulo='Comparación de métricas por modelo con desviación estándar'):
    """
    Genera un gráfico de barras con barras de error (desviación estándar).

    Parámetros:
    - df: DataFrame que contiene los datos
    - modelos_col: nombre de la columna con los nombres de los modelos (default: 'Modelo')
    - metricas: lista con los nombres de las columnas de métricas (default: ['MAE_Mean', 'MSE_Mean'])
    - stds: lista con los nombres de las columnas de desviación estándar correspondientes (default: ['MAE_STD', 'MSE_STD'])
    - titulo: string para el título del gráfico
    """
    # Copiar solo las columnas necesarias
    df_plot = df[[modelos_col] + metricas + stds].copy()

    # Reemplazar NaNs en std por 0
    for std_col in stds:
        df_plot[std_col] = df_plot[std_col].fillna(0)

    # Configuración de posiciones
    x = df_plot[modelos_col]
    x_pos = np.arange(len(x))
    width = 0.35

    # Crear figura
    fig, ax = plt.subplots(figsize=(12, 6))

    # Dibujar cada barra de métrica
    for i, (metrica, std) in enumerate(zip(metricas, stds)):
        offset = (-1)**i * width / 2  # alternar izquierda/derecha
        ax.bar(x_pos + offset,
               df_plot[metrica],
               width=width,
               yerr=df_plot[std],
               capsize=5,
               label=metrica.replace('_Mean', ''))

    # Etiquetas
    ax.set_xticks(x_pos)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.set_ylabel('Valor')
    ax.set_title(titulo)
    ax.legend()
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.savefig(f"{titulo}.png")
    plt.show()


In [ ]:
# Mostramos métricas para HPC
plot_metricas_con_error(final_df, titulo="Comparación de modelos para Household Power Consumption")

### Barajando los mejores modelos

Una vez seleccionado el modelo que queremos comprobar, podemos pasar a ejecutar un experimento: comprobar si el positional encoding está realmente ayudando a aportar valor al modelo. Para ello, podemos realizar un pequeño experimento en el cual, mezclar los valores de entrada de referencia para el decoder, y ver si así se obtienen resultados similares a introducirlos originales.

Lo que buscamos es ver si la información añadida por el PE realmente ayuda al modelo a aprender información local, o si por consiguiente, se está desaprovechando cómputo a cambio leves mejoras o incluso empeoramiento.

Al igual que en el apartado anterior, procederemos a la lectura de resultados de 4 modelos:
- Estadísticos + lags original
- Estadísticos + lags desordenado
- Informer original
- Informer desordenado


In [ ]:
# Ficheros con sus nombres de modelo
model_files = {
    "WinStatFlex": "Experimentos/HPC_ALL_PE_v2.csv",
    "WinStatFlex (Shuffled)": "Experimentos/HPC_ALL_PE_v2_Shuffled.csv",
    "Informer": "Experimentos/metricas_InformerVanilla_hpc_informer_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_hpc_informer.csv",
    "Informer (Shuffled)": "Experimentos/metricas_InformerVanilla_hpc_informer_shuffled_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_hpc_informer_shuffled.csv",
    "No PE": "Experimentos/HPC_No_PE.csv",
    "WinStatTPE": "Experimentos/metricas_InformerVanilla_tpe_weighted_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tpe_weighted.csv",
    "WinStatTPE (Shuffled)": "Experimentos/metricas_InformerVanilla_tpe_weighted_shuffled_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tpe_weighted_shuffled.csv"#,
    #"PE (Fixed)": "Experimentos/metricas_InformerVanilla_hpc_pe_informer_HPCm_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebfixed_dtTrue_mxTrue_test_hpc_pe.csv"
}

dfs = []

for model_name, file_path in model_files.items():
    df = pd.read_csv(file_path)
    #print(file_path)
    df['Modelo'] = model_name  # Añadir columna del modelo

    # Pivotar valores de Mean y STD
    df_mean = df.pivot(index='Modelo', columns='Metric', values='Mean')
    df_std = df.pivot(index='Modelo', columns='Metric', values='STD')

    # Renombrar columnas para distinguir Mean y STD
    df_mean.columns = [f"{col}_Mean" for col in df_mean.columns]
    df_std.columns = [f"{col}_STD" for col in df_std.columns]

    # Combinar ambos
    df_combined = pd.concat([df_mean, df_std], axis=1)

    dfs.append(df_combined)

# Unir todos los modelos
final_df = pd.concat(dfs)

# Resetear índice y ordenar por MSE_Mean
final_df.reset_index(inplace=True)
final_df = final_df.sort_values(by="MSE_Mean")

# Mostramos tabla y gráfico
display(final_df)
plot_metricas_con_error(final_df, titulo="Comparación de modelos para Household Power Consumption (con barajado)")

Podemos apreciar, claramente, que el impacto de mezclar la información de referencia en el encoder afecta de manera considerable a la alternativa que usa ventana de estadísticos, ya que se está rompiendo considerablemente la estructura temporal de los datos. Es un resultado razonable, pues mezclar la información dentro de las ventanas está alterando gravemente la información contenida localmente.

Sin embargo, cuando realizamos el mismo procedimiento con el modelo Informer original, la diferencia es muy poco notable: pasamos apenas de 0.532 de MSE a 0.548. En el caso de las alternativas basadas en estadísticos, se pasa de 0.46 a 0.78, reflejando un empeoramiento significativo en los resultados a diferencia del modelo original.

También se ha evaluado el modelo basado en TPE, debido a su menor varianza y cercanía a los estadísticos. En dicho caso, pasamos de 0.463 a 0.85, siendo aún más notable la diferencia.

Por tanto, podríamos asumir que la información fija aportada por el PE sinusoidal de Informer no está añadiendo suficiente riqueza en datos de larga extensión como este dataset. Si bien, sí que mejora considerablemente el resultado contra un modelo sin ningún tipo de codificación (NoPe En el gráfico), la información añadida es puramente global, y en secuencias de esta longitud, podemos observar cambios de comportamiento a nivel local que pueden ser interesantes de tener en cuenta.

A primera vista, el PE propuesto en ALL_PE_Weighted podría parecer superior, ya que estamos aprovechando dicha información local de utilidad. Sin embargo, tambiém debemos tener en cuenta el encoding basado en TPE, pues ofrece resultados también muy competitivos, y en otros datasets, podría ofrecer un buen resultado, ya que parece ser más estable en sucesivas ejecuciones (menor varianza), y tiene una inferencia más eficiente en tiempo. 

Por lo tanto, será también evaluado con el resto de datasets restantes, debido a que las diferencias en HPC han sido prácticamente marginales.

## Evaluación mediante otros datasets

Una vez definidos los encodings a probar, `TPE` y `All_PE_Weighted`, procederemos a evaluar el rendimiento sobre otros 3 datasets conocidos en el estado del arte:
- **ETTh1**, el dataset de funcionamiento de un sistema eléctrico referenciado en el paper original de Informer, y el cual fue usado para su análisis de rendimiento.
- **Yellow Tripdata**, un conjunto de datos que recopila información acerca de viajes en taxi y su coste asociado. Resulta de interés debido a su posible estacionalidad en horas nocturnas y fines de semana, donde los precios son mayores, e identificar otros patrones (por ejemplo, hora de entrada y salida laboral, etc.). En adelante, para mayor comodidad, lo referenciaremos como `Taxi`.
- **TINA**, cuyo acrónimo proviene de Time-series Industrial Anomaly, donde la gran tarea de interés reside en la detección anomalías y patrones en las mismas.

### ETTh1

Comenzaremos por el dataset de menor tamaño, ETT, el cual es ampliamente utilizado en el estado del arte para la comparativa entre nuevos modelos. Aunque su extensión es bastante reducida, nos permitirá ver el rendimiento del modelo que estamos proponiendo en otro tipo de series, y ver si es lo suficientemente elástico como para adaptarse a diferentes características.

La configuración usada será la misma que en el paper original de Informer para establecer las mismas condiciones. En cuanto a la configuración de la ventana, cuando corresponde, y la longitud de la predicción, se ha establecido:

- Ventana: 24
- Longitud de secuencia: 96
- Longitud de contexto: 48
- Longitud de predicción: 24 (1 día)

A continuación, importaremos los resultados y los graficaremos de la misma forma que el apartado anterior.

In [ ]:
# Ficheros con sus nombres de modelo
model_files = {
    "WinStatFlex": "Experimentos/metricas_InformerVanilla_etth1_all_pe_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_all_pe.csv",
    #"WinStatFlex (Shuffled)": "Experimentos/metricas_InformerVanilla_etth1_all_pe_shuffled_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_all_pe_shuffled.csv",
    "Informer": "Experimentos/metricas_InformerVanilla_etth1_informer_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_informer.csv",
    #"Informer (Shuffled)": "Experimentos/metricas_InformerVanilla_etth1_informer_shuffled_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_informer_shuffled.csv",
    "PE (sin/cos)": "Experimentos/metricas_InformerVanilla_etth1_fixed_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el3_dl2_df2048_atfull_fc5_ebfixed_dtTrue_mxTrue_test_etth1_fixed.csv",
    "No PE": "Experimentos/metricas_InformerVanilla_etth1_nope_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_nope.csv",
    "WinStatTPE": "Experimentos/metricas_InformerVanilla_etth1_tpe_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_tpe.csv",
    #"WinStatTPE (Shuffled)": "Experimentos/metricas_InformerVanilla_etth1_tpe_shuffled_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_tpe_shuffled.csv"
}

dfs = []

for model_name, file_path in model_files.items():
    df = pd.read_csv(file_path)
    #print(file_path)
    df['Modelo'] = model_name  # Añadir columna del modelo

    # Pivotar valores de Mean y STD
    df_mean = df.pivot(index='Modelo', columns='Metric', values='Mean')
    df_std = df.pivot(index='Modelo', columns='Metric', values='STD')

    # Renombrar columnas para distinguir Mean y STD
    df_mean.columns = [f"{col}_Mean" for col in df_mean.columns]
    df_std.columns = [f"{col}_STD" for col in df_std.columns]

    # Combinar ambos
    df_combined = pd.concat([df_mean, df_std], axis=1)

    dfs.append(df_combined)

# Unir todos los modelos
final_df = pd.concat(dfs)

# Resetear índice y ordenar por MSE_Mean
final_df.reset_index(inplace=True)
final_df = final_df.sort_values(by="MSE_Mean")

# Mostramos tabla y gráfico
display(final_df)
plot_metricas_con_error(final_df, titulo=f"Comparación de modelos para ETTh$_1$")

Podemos apreciar resultados muy diferentes a los vistos para una secuencia más larga:

- El rendimiento de All_PE siguen siendo el ganador, pero esta vez, con mayor diferencia, siendo esta vez de 0.04, que es 40 veces mayor a la diferencia existente en HPC. Además, la varianza de TPE es ahora mayor a la de ALL_PE, tal y como se puede verificar tanto numéricamente como en la barra de error.
- El rendimiento de Informer, curiosamente, parece ser más cercano en este conjunto. Podemos ver que, además, cuando realizamos el mezclado, ahora es el modelo de Informer el que más impacto aprecia en sus resultados. De alguna manera, para secuencias más cortas, una simple codificación sinusoidal parece codificar adecuadamente la información.
- En el caso de los dos modelos propuestos, pero mezclados, el empeoramiento sigue siendo notable como hasta ahora.

Por tanto, este apartado nos arroja un resultado bastante interesante, y es que todo apunta que, a secuencias más largas, y de mayor complejidad, el PE tradicional basado en senos y cosenos tiene grandes carencias, potencialmente por la falta de aprovechamiento de la localidad, ya que series de gran tamaño podemos encontrar diferentes subsecuencias a las que debemos adaptarnos. Pero, cuando la serie es más corta, como esta, dicha información podría no ser tan variable, y de ahí que un enfoque global como Informer puedea seguir manteniendose cercano a los mecanismos locales.

También podríamos sumar el impacto de la profundidad de las arquitecturas y estructuras de datos: a mayor tamaño de entrada, mayor número de parámetros, y por ende, más dificultad de propagación del aprendizaje del encoding a través de la arqutectura. Sin embargo, para verificarlo, debemos seguir evaluando más datasets.

In [ ]:
# Ficheros con sus nombres de modelo
model_files = {
    "WinStatFlex": "Experimentos/metricas_InformerVanilla_ETTh2_new_allpe_informer_ETTh2_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_ETTh2_new_allpe.csv",
    #"WinStatFlex (Shuffled)": "Experimentos/metricas_InformerVanilla_etth1_all_pe_shuffled_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_all_pe_shuffled.csv",
    "Informer": "Experimentos/metricas_InformerVanilla_ETTh2_new_informer_informer_ETTh2_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_ETTh2_new_informer.csv",
    #"Informer (Shuffled)": "Experimentos/metricas_InformerVanilla_ETTh2_newinformer_shuffled_informer_ETTh2_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_ETTh2_newinformer_shuffled.csv",
    "PE (sin/cos)": "Experimentos/metricas_InformerVanilla_ETTh2_new_fixed_informer_ETTh2_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebpe_dtTrue_mxTrue_test_ETTh2_new_fixed.csv",
    "No PE": "Experimentos/metricas_InformerVanilla_ETTh2_new_nope_informer_ETTh2_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_ETTh2_new_nope.csv",
    "WinStatTPE": "Experimentos/metricas_InformerVanilla_ETTh2_new_tpe_informer_ETTh2_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_ETTh2_new_tpe.csv",
    #"WinStatTPE (Shuffled)": "Experimentos/metricas_InformerVanilla_etth1_tpe_shuffled_informer_ETTh1_ftM_sl96_ll48_pl24_win24_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_etth1_tpe_shuffled.csv"
}

dfs = []

for model_name, file_path in model_files.items():
    df = pd.read_csv(file_path)
    #print(file_path)
    df['Modelo'] = model_name  # Añadir columna del modelo

    # Pivotar valores de Mean y STD
    df_mean = df.pivot(index='Modelo', columns='Metric', values='Mean')
    df_std = df.pivot(index='Modelo', columns='Metric', values='STD')

    # Renombrar columnas para distinguir Mean y STD
    df_mean.columns = [f"{col}_Mean" for col in df_mean.columns]
    df_std.columns = [f"{col}_STD" for col in df_std.columns]

    # Combinar ambos
    df_combined = pd.concat([df_mean, df_std], axis=1)

    dfs.append(df_combined)

# Unir todos los modelos
final_df = pd.concat(dfs)

# Resetear índice y ordenar por MSE_Mean
final_df.reset_index(inplace=True)
final_df = final_df.sort_values(by="MSE_Mean")

# Mostramos tabla y gráfico
display(final_df)
plot_metricas_con_error(final_df, titulo=f"Comparación de modelos para ETTh$_2$")

### Yellow Trip Data (Taxi)

A continuación, es el turno de un dataset de extensión similar a HPC, por lo que se usarán los mismos parámetros que en este para la evaluación del rendimiento. De la misma forma, se han recopilado resultados para All_PE, TPE e Informer.

- Ventana: 60
- Longitud de secuencia: 180
- Longitud de contexto: 60
- Longitud de predicción: 60 

Para facilitar la visualización de los datos, al ser valores muy pequeños, se ha creado una función adicional que cree el gráfico de barras en dos ejes diferentes, uno para MSE y otro para MAE, ya que los valore se encuentran en el orden de 10-5.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_metricas_por_separado(df,
                               modelos_col='Modelo',
                               metricas=['MAE_Mean', 'MSE_Mean'],
                               stds=['MAE_STD', 'MSE_STD'],
                               titulos=['MAE por modelo con desviación estándar',
                                        'MSE por modelo con desviación estándar']):
    """
    Genera gráficos de barras separados con barras de error (desviación estándar) para cada métrica.

    Parámetros:
    - df: DataFrame que contiene los datos
    - modelos_col: nombre de la columna con los nombres de los modelos
    - metricas: lista con los nombres de las columnas de métricas
    - stds: lista con los nombres de las columnas de desviación estándar correspondientes
    - titulos: lista de títulos para los gráficos
    """
    df_plot = df[[modelos_col] + metricas + stds].copy()

    # Reemplazar NaNs en std por 0
    for std_col in stds:
        df_plot[std_col] = df_plot[std_col].fillna(0)

    x = df_plot[modelos_col]
    x_pos = np.arange(len(x))
    width = 0.5
    colors = ["tab:blue", "tab:orange"]
    # Crear una figura por métrica
    for i, (metrica, std) in enumerate(zip(metricas, stds)):
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.bar(x_pos,
               df_plot[metrica],
               width=width,
               yerr=df_plot[std],
               capsize=5,
               color = colors[i])

        ax.set_xticks(x_pos)
        ax.set_xticklabels(x, rotation=45, ha='right')
        ax.set_ylabel('Valor')
        ax.set_title(titulos[i])
        ax.grid(True, axis='y', linestyle='--', alpha=0.5)

        plt.tight_layout()
        plt.show()


In [ ]:
# Ficheros con sus nombres de modelo
model_files = {
    "WinStatFlex": "Experimentos/metricas_InformerVanilla_taxi_all_pe_weighted_informer_Taxi_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_taxi_all_pe_weighted.csv",
    "PE (sin/cos)": "Experimentos/metricas_InformerVanilla_taxi_pe_informer_Taxi_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebfixed_dtTrue_mxTrue_test_taxi_pe.csv",
    "Informer": "Experimentos/metricas_InformerVanilla_taxi_informer_informer_Taxi_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_taxi_informer.csv",
    #"Informer (Shuffled)": "Experimentos/metricas_InformerVanilla_taxi_informer_shuffled_informer_Taxi_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_taxi_informer_shuffled.csv",
    "No PE": "Experimentos/metricas_InformerVanilla_taxi_nope_informer_Taxi_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_taxi_nope.csv",
    "WinStatTPE": "Experimentos/metricas_InformerVanilla_taxi_tpe_informer_Taxi_ftM_sl180_ll60_pl60_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_taxi_tpe.csv"
}

dfs = []

for model_name, file_path in model_files.items():
    df = pd.read_csv(file_path)
    #print(file_path)
    df['Modelo'] = model_name  # Añadir columna del modelo

    # Pivotar valores de Mean y STD
    df_mean = df.pivot(index='Modelo', columns='Metric', values='Mean')
    df_std = df.pivot(index='Modelo', columns='Metric', values='STD')

    # Renombrar columnas para distinguir Mean y STD
    df_mean.columns = [f"{col}_Mean" for col in df_mean.columns]
    df_std.columns = [f"{col}_STD" for col in df_std.columns]

    # Combinar ambos
    df_combined = pd.concat([df_mean, df_std], axis=1)

    dfs.append(df_combined)


# Unir todos los modelos
final_df = pd.concat(dfs)

# Resetear índice y ordenar por MSE_Mean
final_df.reset_index(inplace=True)
final_df = final_df.sort_values(by="MSE_Mean")

# Mostramos tabla y gráfico
display(final_df)
plot_metricas_por_separado(final_df, titulos=['Comparación de modelos para Yellow Trip Data - MAE',
                                        'Comparación de modelos para Yellow Trip Data - MSE'])

A la vista de los resultados, podemos extraer las siguientes conclusiones:

- Los resultados pueden encontrarse en una escala bastante pequeña. Esto puede deberse, o bien, a que la amplitud de los valores de la serie son bastante pequeños, o bien, que se trata de una serie fácilmente aprendible, ya sea por mantener una tendencia clara o bien una estacionalidad sencilla de modelar. Tratándose de una recopilación de datos sobre carreras de Taxi, esto segundo podría ser la principal causa, ya que los precios son mayores de noche, y además, incrementan también los fines de semana, y en media, los viajes realizados serán bastante similares. Sin embargo, se requiere el estudio de estos datos en mayor profundidad mediante EDA para asegurar.

- La mejor codificación, tanto para MAE como MSE, es la basada en TPE. Anteriormente, vimos que ofrecía valores similares a usar ALL_PE, pero en este caso, posiblemente su menor variabilidad, así como su incorporación a la tradicional fórmula sinusoidal de una componente exponencial dependiente de la información, permite tomar una mayor información local para el modelado. All_PE, sin embargo, sigue siendo una sólida alternativa, con un resultado bastante similar, aunque con prácticamente el doble de error de TPE.
- A diferencia de los datasets anteriores, el modelo original de Informer no se distancia tanto en cuanto a resultados, y ocupa la tercera posición. Si procedemos además a evaluar lo que ocurre cuando se procede a desordenar la entrada del decoder, podemos apreciar que el error se duplica, demostrando que sí es capaz de recoger parte de la infomación temporal de los datos. 
- Por último, no usar ningún tipo de codificación no aporta prácticamente ninguna ventaja, y se trata del peor caso de todos. Es además el que mayor varianza recoge en sus resultados, ya que como no se agrega información adicional a los datos, el resultado es muy dependiente de inicialización.


Por lo tanto, en este caso, podemos afirmar de manera evidente el correcto funcionamiento de TPE con respecto al resto de alternativas.

### Dataset TINA

Hasta ahora, se han ejecutado datasets de larga extensión temporal, como es el caso del anterior de Yellow Trip, o el asociado al consumo eléctrico HPC. Sin embargo, en cuanto número de variables, estos conjuntos de datos no tenían más de 7 variables. Pero, para probar la afectividad de los métodos, es necesario probar con datasets de complejidad en cuanto a variables. En este caso, TINA, cuyo conjunto de datos recopila mediciones de diferentes sensores de una acerería de ArcelorMittal (https://dasci.es/opendata/tina-time-series-industrial-anomaly-dataset/) es un buen candidato para medir la eficiencia de las codificaciones evaluadas cuando la complejidad reside sobre todo en el número de variables medidas por instante.

Quedándonos únicamente con las variables numéricas, tenemos en total 103 características a estimar mediante el modelo, por lo que dado el cambio de tipología de problema, podemos apreciar resultados bastante diferentes a los anteriores.

In [ ]:
# Ficheros con sus nombres de modelo
model_files = {
    "WinStatFlex": "Experimentos/metricas_InformerVanilla_tina_all_pe_opt_informer_TINA_ftM_sl288_ll96_pl48_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tina_all_pe_opt.csv",
    "Informer": "Experimentos/metricas_InformerVanilla_tina_informer_opt_informer_TINA_ftM_sl288_ll96_pl48_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tina_informer_opt.csv",
    "PE (sin/cos)": "Experimentos/metricas_InformerVanilla_tina_pe_fixed_informer_TINA_ftM_sl288_ll96_pl48_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebfixed_dtTrue_mxTrue_test_tina_pe_fixed.csv",
    #"Informer (shuffled)":"Experimentos/metricas_InformerVanilla_tina_informer_opt_shuffled_informer_TINA_ftM_sl288_ll96_pl48_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tina_informer_opt_shuffled.csv",
    "No PE": "Experimentos/metricas_InformerVanilla_tina_not_pe_informer_TINA_ftM_sl288_ll96_pl48_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tina_not_pe.csv",
    "WinStatTPE": "Experimentos/metricas_InformerVanilla_tina_tpe_opt_informer_TINA_ftM_sl288_ll96_pl48_win60_dm512_nh8_el2_dl1_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_test_tina_tpe_opt.csv",
}

dfs = []

for model_name, file_path in model_files.items():
    df = pd.read_csv(file_path)
    #print(file_path)
    df['Modelo'] = model_name  # Añadir columna del modelo

    # Pivotar valores de Mean y STD
    df_mean = df.pivot(index='Modelo', columns='Metric', values='Mean')
    df_std = df.pivot(index='Modelo', columns='Metric', values='STD')

    # Renombrar columnas para distinguir Mean y STD
    df_mean.columns = [f"{col}_Mean" for col in df_mean.columns]
    df_std.columns = [f"{col}_STD" for col in df_std.columns]

    # Combinar ambos
    df_combined = pd.concat([df_mean, df_std], axis=1)

    dfs.append(df_combined)

# Unir todos los modelos
final_df = pd.concat(dfs)

# Resetear índice y ordenar por MSE_Mean
final_df.reset_index(inplace=True)
final_df = final_df.sort_values(by="MSE_Mean")

# Mostramos tabla y gráfico
display(final_df)
plot_metricas_con_error(final_df, titulo=f"Comparación de modelos para TINA")

Como puede observarse, en este caso Informer logra un rendimiento notablemente superior en comparación con el resto de métodos evaluados:

- El modelo TPE presenta un desempeño relativamente cercano, con una diferencia media en el error cuadrático medio (MSE) de apenas $10^{-2}$. No obstante, esta pequeña variación podría considerarse suficiente para afirmar que la versión original de Informer demuestra ventaja.
- Por otro lado, el modelo All_PE obtiene resultados aún más próximos a los de TPE. En este caso, logra una menor varianza, aunque la diferencia podría deberse a variabilidad estadística más que a una ventaja estructural clara del encoding.

- Tal como era esperable, la opción de no emplear ningún tipo de codificación posicional resulta ser la menos efectiva. Pero sí que es importante destacar que en este caso no se distancia tanto de las soluciones que sí incorporan información posicional, debido a que la complejidad de la serie dificulta su predicción a largo plazo y las diferencias se acortan.

Sin duda, el dato más llamativo que podemos encontrar son los resultados del modelo original de Informer, pero con mezclado en su entrada: ofrece un mejor rendimiento que el resto de modelos. Se trata de un resultado poco intuitivo, ya que romper a propósito la temporalidad de los datos parece ir en contra de lo razonable. Buscar una respuesta a este comportamiento es bastante complicado; tal vez, podría ser furto del azar, ya que el mezclado es un proceso totalmente estocástico. Pero, al ser ejecutado en varias ocasiones para hacer el promedio, esta hipótesis puede prácticamente descartarse, ya que a mayor número de ejecuciones, más nos acercamos al rendimiento real del modelo. Posiblemente, este fenómeno sea atribuible a las propiedades del conjunto de datos en sí:

- El conjunto de datos recoge gran cantidad de valores de diferentes sensores y localizaciones, las cuales han sido anonimizadas, por lo que es posible que algunas de estas propiedades fueran dependientes directamente entre sí, y provoquen redundancia que dificulte a los modelos codificados a extraer información útil.
- Que las variables no tengan un comportamiento modelable con las características dadas. Puede que dependa de factores externos ajenos a los datos recogidos, los cuales no puedan ser directamente deducibles de los datos dados, y por tanto, dificulten el modelado correcto de la serie. O bien, que directamente su comportamiento no sea modelable al seguir algún tipo de distribución aleatoria o muy ruidosa: en ese caso, mezclar podría no afectar al resultado con tanto impacto como en los conjuntos anteriores debido a que la información temporal no aporta información útil sobre el comportamiento de cada variable.


**Sin lugar a dudas, este caso requiere especial atención en la próxima reunión.**

## Resumen final

Para resumir las claves de manera muy breve, se condensan aquí las principales conclusiones que, a mi juicio, se pueden extraer de los conjuntos de datos evaluados acerca de las codificaciones:

- Cuanto mayor es la extensión de la serie, menor impacto posee la codificación empleada en los resultados. Se reducen las distancia entre los modelos con y sin codificación, e Informer comienza a no alejarse tanto de los modelos diseñados.
- En general, los dos modelos escogidos, TPE y ALL_PE (que hace uso de pesos de ponderación para cada tipo de encoding) son los que ofrecen los mejores resultados en la mayoría de casos, siendo TPE más estable numéricamente en sucesivas ejecuciones.
- Los conjuntos de mayor extensión, como TINA y PLP, ofrecen comportamiento bastante anómalos frente a los anteriores, obteniendo TINA un mejor rendimiento cuando se mezcla su entrada de contexto al decoder, y PLP imposible de ejecutar con los recursos actuales.

# Parte 2: Estudio del valor de ventana para algoritmos ALL PE y TPE

En este apartado, examinaremos el impacto del tamaño de ventana en nuestros conjuntos de datos de menor tamaño, contenidos en paper original de Informer. El objetivo consiste en ver si existe alguna relación con los valores de ventana probados con la longitud de secuencia del experimento, de forma que podamos establecer intervalos adecuados para datasets de mayor tamaño que no permitan evaluar una búsqueda de parámetros exhaustiva. El efecto deseable que se desea encontrar es algún tipo de regla heurística que nos permita realacionar ambos valores, para lo cual, tendremos que evaluar diversos datasets de longitud breve que puedan entrenarse con diferentes tamaños y con cierta garantía estadística, realizando varias ejecuciones para cada prueba.


Los datasets escogidos son los empleados en el paper original de Informer (https://arxiv.org/abs/2012.07436): ETTh1, ETTh2, ETTm1, Weather y ECL, con la incorporación de ETTm2, el cual podemos encontrar en los archivos del repositorio pero no fue utilizado en las pruebas experimentales de Informer por algún motivo que se desconoce. Por tanto, resulta interesante probar su desempeño con la alternativa propuesta.

El escenario de pruebas escogido cumple las siguientes características:

- El tamaño de secuencia es el recomendado por las ejecuciones realizadas en el paper de Informer, siguiendo la estructura de la tabla. Por ejemplo, en el caso de ETTh, el tamaño de secuencia de 96, token de contexto de 48, y longitud de predicción de 24.
- Las pruebas se realizan 10 veces por tamaño de ventana, de manera que exista estabilidad numérica en los resultados y la curva muestre un comportamiento más cercano al real, tratando de suavizar efectos provocados por ruido.
- Los modelos evaluados son ALL_PE y TPE, ya que son los que mayor rendimiento exhibieron en las pruebas realizadas sobre datasets de mayor tamaño, y ambos requieren de un ajuste de ese parámetro.



In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

A continuación, podemos encontrar el código empleado para la representación de los datos. Se procede a realizar la lectura de los ficheros asociados a cada tipo de ventana y algoritmo de los mencionados, y se procede a realizar un gráfico que muestre el comportamiento de cada algoritmo con respecto al tamaño de ventana y el error cuadrático medio.

In [ ]:
def extract_mse_from_csv(filepath):
    """Extrae el valor de MSE (Mean) desde un archivo CSV."""
    df = pd.read_csv(filepath)
    row = df[df['Metric'] == 'MSE']
    if not row.empty:
        return float(row['Mean'].values[0])
    return None


def collect_mse_by_window(dataset, log_folder, start_window, end_window):
    """Recoge los valores MSE por tamaño de ventana para all_pe_weighted y tpe."""
    results_pe, results_tpe = {}, {}

    for window in range(start_window, end_window + 1):
        pe_pattern = os.path.join(
            log_folder, f"{dataset}_ALL_PE", f"*win{window}*.csv")
        tpe_pattern = os.path.join(
            log_folder, f"{dataset}_TPE", f"*win{window}*.csv")

        pe_files = glob.glob(pe_pattern)
        tpe_files = glob.glob(tpe_pattern)

        if pe_files:
            mse_pe = extract_mse_from_csv(pe_files[0])
            if mse_pe is not None:
                results_pe[window] = mse_pe

        if tpe_files:
            mse_tpe = extract_mse_from_csv(tpe_files[0])
            if mse_tpe is not None:
                results_tpe[window] = mse_tpe

    return dict(sorted(results_pe.items())), dict(sorted(results_tpe.items()))


def print_top_results(data, label, top_n=5):
    """Imprime los top N resultados con menor MSE."""
    print(f"\nTop {top_n} resultados para {label}:")
    sorted_items = sorted(data.items(), key=lambda x: x[1])
    for i, (window, mse) in enumerate(sorted_items[:top_n], 1):
        print(f"{i}. Window: {window} - MSE: {mse:.6f}")


def annotate_top_points(x, y, top_n, label_color):
    """Anota los top N puntos mínimos en la curva."""
    sorted_points = sorted(zip(x, y), key=lambda x: x[1])[:top_n]
    for win, val in sorted_points:
        plt.annotate(f"{win}", xy=(win, val), xytext=(win, val + 0.01),
                     ha='center', fontsize=12, fontweight='bold', color=label_color,
                     arrowprops=dict(arrowstyle='->', color=label_color, lw=0.8))


def plot_curvas_mse(pe_data, tpe_data, dataset, top_n=5):
    """Grafica las curvas de MSE para PE y TPE, con anotaciones."""
    x_pe = list(pe_data.keys())
    y_pe = list(pe_data.values())
    x_tpe = list(tpe_data.keys())
    y_tpe = list(tpe_data.values())

    plt.figure(figsize=(10, 6))
    plt.plot(x_pe, y_pe, label="all_pe_weighted", marker="o")
    plt.plot(x_tpe, y_tpe, label="tpe", marker="s")

    annotate_top_points(x_pe, y_pe, top_n=top_n, label_color="blue")
    annotate_top_points(x_tpe, y_tpe, top_n=top_n, label_color="green")

    plt.xlabel("Window Size")
    plt.ylabel("Mean MSE")
    plt.title(f"Mean MSE vs Window Size ({dataset})")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    #plt.savefig(f"mean_mse_vs_window_{dataset}.png")
    plt.show()

## ETTh1

En primer lugar, volveremos a evaluar el primer dataset asociado a timestamps horarios, que ya probamos en el apartado anterior, para estudiar cuál es valor ideal del tamaño de ventana. Usando un tamaño de ventana de mínimo 3, y un máximo de hasta 96, coincidiendo con la longitud de secuencia.

- Longitud de secuencia: 96
- Longitud de token de contexto: 48
- Longitud de predicción: 24

Podemos obtener los resultados que se aprecian en el siguiente gráfico:

In [ ]:

DATASET = "ETTh1"
LOG_FOLDER = "Experimentos/EstudioVentana"
WINDOW_RANGE = (3, 96)


start, end = WINDOW_RANGE
results_pe, results_tpe = collect_mse_by_window(DATASET, LOG_FOLDER, start, end)
plot_curvas_mse(results_pe, results_tpe, DATASET)

Podemos apreciar que ambos algoritmos sufren un comportamiento bastante irregular, y cercanos entorno a un valor medio concreto: sobre 0.54  en el caso de TPE, y 0.48 en el caso de ALL_PE. Sin embargo, aunque su tendencia es bastante plana, podemos apreciar que si existe una mejora razonable en los resultados cuando nos acercamos al entorno de valores entre el 24-30. En ese intervalo, se ubican los 5 mejores resultados de ambos algoritmos (a expcepción del tamaño 73 para TPE, pero podría ser un punto ruidoso).

Esto podría indicarnos que un valor dentro de dicho intervalo nos permite mejorar los resultados. En ambos casos, resulta bastante evidente que el 24 parece ser el "número mágico" que estábamos buscando. Curiosamente, se trata exactamente de un cuarto del valor de longitud de secuencia, ya que: $$96\%4 = 0$$

Por lo tanto, podríamos extraer a priori una heurística que nos indica que escoger como tamaño de ventana 1/4 de la longitud de secuencia parece razonable.

## ETTh2

Una vez evaluada la primera variante, es turno de ejecutar ETTh2. Al tener características similares a la primera, usaremos exactamente los mismos parámetros:
- Longitud de secuencia: 96
- Longitud de token de contexto: 48
- Longitud de predicción: 24

Las condiciones de los valores probados son los mismos: desde tamaño 3 hasta 96, coincidiendo con el tamaño de secuencia.

In [ ]:

DATASET = "ETTh2"
LOG_FOLDER = "Experimentos/EstudioVentana"
WINDOW_RANGE = (3, 96)


start, end = WINDOW_RANGE
results_pe, results_tpe = collect_mse_by_window(DATASET, LOG_FOLDER, start, end)
plot_curvas_mse(results_pe, results_tpe, DATASET)

Observando la gráfica anterior, podemos ver que en este caso, las distancias entre ambos algoritmos es menor, ofreciendo un resultado similar. Pero los más interesante se puede observar en la forma de las curvas; el comportamiento es bastante más sencillo de interpretar que el caso anterior, e indica grandes ganancias en rendimiento de MSE de nuevo en el entorno de valores de 23 a 30 para ambos algoritmos.

Esto refuerza nuestra hipótesis del 1/4 de manera bastante contundente, pero a continuación, tendremos que probar en datasets de mayor granularidad.

## ETTm1

Ahora, procedemos a realizar la misma ejecución pero empleando la versión minuto a minuto del primer transformador.

In [ ]:

DATASET = "ETTm1"
LOG_FOLDER = "Experimentos/EstudioVentana"
WINDOW_RANGE = (3, 96)


start, end = WINDOW_RANGE
results_pe, results_tpe = collect_mse_by_window(DATASET, LOG_FOLDER, start, end)
plot_curvas_mse(results_pe, results_tpe, DATASET)

De nuevo, encontramos un patrón similar a la serie sampleada por horas, pero con una ligera tendencia a menores tamaños de ventana. Aun así, los valores cercanos a 1/4, es decir, 24, siguen incluidos en el intervalo. El valor 72, curiosamente, destaca como un valor inferior, al igual que lo hizo 73 en la variante horaria. Sin embargo, es preferible escoger un valor de la región más a la izquierda, un flat minima, pues posiblemente generalizará mejor que el 72, que es un sharp minima.

## ETTm2

Aplicaremos las mismas condiciones para la variante minuto a minuto de ETT2:

- Longitud de secuencia: 96
- Longitud de token de contexto: 48
- Longitud de predicción: 24

In [ ]:

DATASET = "ETTm2"
LOG_FOLDER = "Experimentos/EstudioVentana"
WINDOW_RANGE = (3, 96)


start, end = WINDOW_RANGE
results_pe, results_tpe = collect_mse_by_window(DATASET, LOG_FOLDER, start, end)
plot_curvas_mse(results_pe, results_tpe, DATASET)

El comportamiento arrojado por este dataset es bastante diferente a los anteriores. Si analizamos gráficamente su comportamiento (como ya se hizo en la memoria del TFM), podemos apreciar que la tendencia seguida por esta serie temporal siempre ha sido diferente a las otras 3, e incluso fue descartado su uso en la paper original de Informer. Si observamos, los valores destacados con un mayor rendimiento tienden a distribuirse de manera prácticamente uniforme en la mitad derecha de la gráfica, no modificando drásticamente su comportamiento, si bien parece que comienza una ligera tendencia de empeoramiento a partir del valor 80.

Debido a este extraño comportamiento, en este dataset no podríamos aplicar la regla del cuarto, ya que potencialmente fomentada por una mayor complejidad, esta serie se beneficia de tamaños de ventana mayores. Para no escoger un valor demasiado grande, considero que en este caso el valor central, 48, sería más que suficiente, ya que se trata de un valor intermedio entre todos los valores emergentes.

## Weather Dataset

A continuación, se muestra el mismo experimento para el dataset Weather

In [ ]:

DATASET = "Weather"
LOG_FOLDER = "Experimentos/EstudioVentana"
WINDOW_RANGE = (3, 96)


start, end = WINDOW_RANGE
results_pe, results_tpe = collect_mse_by_window(DATASET, LOG_FOLDER, start, end)
plot_curvas_mse(results_pe, results_tpe, DATASET)